In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from utils.gtsrb_dataset import GTSRBDataset
from models.tsr_cnn import TSRNet
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm 

In [6]:
from utils.train_tsr import train_tsr
from models.tsr_cnn import TSRNet
from torch.utils.data import DataLoader
from torchvision import transforms
from utils.gtsrb_dataset import GTSRBDataset

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Datasets & loaders
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_dataset = GTSRBDataset(csv_file="./data/Train.csv", root_dir="./data/", transform=transform)
test_dataset  = GTSRBDataset(csv_file="./data/Test.csv", root_dir="./data/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model
model = TSRNet(num_classes=43).to(device)

# Train
model, best_acc = train_tsr(model, train_loader, test_loader, device, num_epochs=5)


Epoch 1/5:   0%|          | 0/613 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x16384 and 4096x256)

In [ ]:
from utils.train_tsr import train_tsr
from models.tsr_cnn import TSRNet
from torch.utils.data import DataLoader
from torchvision import transforms

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Datasets & loaders
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_dataset = GTSRBDataset(csv_file="./data/Train.csv", root_dir="./data/", transform=transform)
test_dataset  = GTSRBDataset(csv_file="./data/Test.csv",  root_dir="./data/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Train samples: {len(train_dataset)} | Test samples: {len(test_dataset)}")
# Model
model = TSRNet(num_classes=43).to(device)

# Train
model, best_acc = train_tsr(model, train_loader, test_loader, device, num_epochs=5)


In [ ]:
from models.tsr_cnn import TSRNet
from utils.attack_eval import evaluate_clean, evaluate_fgsm, evaluate_pgd, evaluate_patch
from utils.attacks import train_adversarial_patch
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load TSR model
model = TSRNet(num_classes=43).to(device)
ckpt = torch.load("results/checkpoints/tsr_best_model.pth", map_location=device)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

# Evaluate clean accuracy
clean_acc = evaluate_clean(model, test_loader, device)
print("Clean Accuracy:", clean_acc)

# FGSM
eps = 0.03
adv_acc, asr = evaluate_fgsm(model, test_loader, device, eps)
print(f"FGSM Adv Acc: {adv_acc:.2f}%, ASR: {asr:.2f}%")

# PGD
adv_acc, asr = evaluate_pgd(model, test_loader, device, eps, alpha=0.007, iters=10)
print(f"PGD Adv Acc: {adv_acc:.2f}%, ASR: {asr:.2f}%")

# Train and evaluate patch attack
patch = train_adversarial_patch(model, train_loader, device, num_epochs=1, patch_size=0.18, lr=0.05)
res = evaluate_patch(model, test_loader, patch, device, patch_size=0.18)
print("Patch attack eval:", res)


In [ ]:
from utils.train_autoencoder import train_autoencoder
from models.autoencoder import Autoencoder, load_checkpoint
from utils.visualization import save_sample  # optional
from torch.utils.data import DataLoader
from torchvision import transforms

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Datasets & loaders
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_dataset = GTSRBDataset(csv_file="./data/Train.csv", root_dir="./data/", transform=transform)
test_dataset  = GTSRBDataset(csv_file="./data/Test.csv",  root_dir="./data/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Cell: train
model, best_loss, best_ckpt = train_autoencoder(
    train_loader, test_loader, device,
    latent_dim=128, num_epochs=5, lr=1e-3, ckpt_dir="results/checkpoints/autoencoder"
)
print("Best checkpoint:", best_ckpt)


In [ ]:
# setup
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from utils.train_unet_watermark import train_unet_watermark
from utils.gtsrb_dataset import GTSRBDataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_dataset = GTSRBDataset(csv_file="./data/Train.csv", root_dir="./data/", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
print("Train samples:", len(train_dataset))

unet, best_loss = train_unet_watermark(
    train_loader, device,
    autoencoder_ckpt="results/checkpoints/autoencoder/autoencoder_best.pth",
    latent_dim=128,
    num_epochs=6,
    lr=1e-4,
    save_watermarked_dir="results/watermarked",
    ckpt_dir="results/checkpoints/unet"
)
print("Finished. Best perceptual loss:", best_loss)



In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# -----------------------------
# 1. Dataset (CSV-based)
# -----------------------------
class GTSRBDatasetCSV(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Use the 'Path' column from CSV
        img_name = os.path.join(self.root_dir, self.df.iloc[idx]["Path"])
        label = int(self.df.iloc[idx]["ClassId"])
        img = Image.open(img_name).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import DataLoader

def generate_watermarked(dataset, output_dir, output_csv, ae_model, unet_model, device, batch_size=32):
    """
    Generate watermarked images using the autoencoder latent vector and UNet model.
    Saves watermarked images and a CSV with paths and labels.

    Args:
        dataset: torch Dataset
        output_dir: directory to save watermarked images
        output_csv: CSV path to save new paths and labels
        ae_model: trained Autoencoder
        unet_model: trained UNetWatermark
        device: 'cuda' or 'cpu'
        batch_size: batch size for DataLoader
    """
    os.makedirs(output_dir, exist_ok=True)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    records = []

    ae_model.eval()
    unet_model.eval()

    with torch.no_grad():
        for batch_idx, (imgs, labels) in enumerate(loader):
            imgs = imgs.to(device)
            labels = labels.to(device)

            # Get latent vectors
            z = ae_model.encoder(imgs)

            # Generate watermarked images
            Iw = unet_model(imgs, z)

            # Save images
            for i in range(imgs.size(0)):
                img_np = ((Iw[i].cpu().permute(1, 2, 0).numpy() * 0.5) + 0.5) * 255
                img_np = img_np.astype("uint8")
                save_name = f"{batch_idx * batch_size + i}.png"
                save_path = os.path.join(output_dir, save_name)
                Image.fromarray(img_np).save(save_path)

                # Record new path and label
                records.append((save_name, labels[i].item()))

    # Save new CSV
    df = pd.DataFrame(records, columns=["Path", "ClassId"])
    df.to_csv(output_csv, index=False)
    print(f"✅ Watermarked images saved to {output_dir} and CSV to {output_csv}")


from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_dataset = GTSRBDatasetCSV(csv_file="./data/Train.csv", root_dir="./data/", transform=transform)
test_dataset  = GTSRBDatasetCSV(csv_file="./data/Test.csv", root_dir="./data/", transform=transform)

output_train_dir = "./data/WatermarkedTrain"
output_test_dir  = "./data/WatermarkedTest"

# Load models
ae_model.load_state_dict(torch.load("./results/checkpoints/autoencoder/autoencoder_best.pth")["model_state"])
unet_ckpt = torch.load("./results/checkpoints/unet/unet_best.pth")
unet_model.load_state_dict(unet_ckpt["model_state"])

# Generate watermarked images
generate_watermarked(train_dataset, output_train_dir, "./data/WatermarkedTrain.csv", ae_model, unet_model, device)
generate_watermarked(test_dataset, output_test_dir, "./data/WatermarkedTest.csv", ae_model, unet_model, device)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision.transforms import InterpolationMode
from PIL import Image
import io

class NoiseLayer(nn.Module):
    def __init__(self):
        super().__init__()
        # learnable noise strength
        self.log_sigma = nn.Parameter(torch.tensor([0.0]))  # initial std = exp(0) = 1
        # learnable jpeg quality (optional, could clamp between 30-100)
        self.jpeg_quality = nn.Parameter(torch.tensor([95.0]))

    def forward(self, x):
        # Gaussian noise
        sigma = self.log_sigma.exp()
        noise = torch.randn_like(x) * sigma
        x_noisy = x + noise
        x_noisy = torch.clamp(x_noisy, -1, 1)

        # JPEG attack simulation
        jpeg_imgs = []
        for img in x_noisy:
            img_pil = TF.to_pil_image((img * 0.5 + 0.5).clamp(0,1))
            buffer = io.BytesIO()
            quality = int(torch.clamp(self.jpeg_quality, 30, 100).item())
            img_pil.save(buffer, format="JPEG", quality=quality)
            img_jpeg = Image.open(buffer)
            img_tensor = TF.to_tensor(img_jpeg) * 2 - 1  # back to [-1,1]
            jpeg_imgs.append(img_tensor)
        x_noisy = torch.stack(jpeg_imgs).to(x.device)

        return x_noisy


In [ ]:
def latent_consistency_loss(z_orig, z_noisy):
    return F.mse_loss(z_noisy, z_orig)


In [ ]:
from torch.utils.data import DataLoader

# Assume train_loader already exists with watermarked images
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

noise_layer = NoiseLayer().to(device)
ae_model.eval()        # freeze AE
unet_model.eval()      # freeze UNet

optimizer = torch.optim.Adam(noise_layer.parameters(), lr=1e-3)
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        
        # Apply watermarked UNet
        with torch.no_grad():
            z = ae_model.encoder(imgs)
            watermarked = unet_model(imgs, z)

        # Apply trainable noise layer
        attacked = noise_layer(watermarked)

        # Encode attacked image
        z_attacked = ae_model.encoder(attacked)

        # Latent consistency loss
        loss = latent_consistency_loss(z, z_attacked)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] | Latent Consistency Loss: {epoch_loss:.6f}")


In [ ]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torch
from torchvision import transforms
import json

# -----------------------------
# Dataset for GTSRB CSV
# -----------------------------
class GTSRBDatasetCSV(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.df.iloc[idx]["Path"])
        label = int(self.df.iloc[idx]["ClassId"])
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label, img_path  # include original path

# -----------------------------
# Transform
# -----------------------------
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

# -----------------------------
# Function to generate noisy images
# -----------------------------
def generate_noisy_images(loader, ae_model, unet_model, noise_layer, output_dir, output_csv, device):
    os.makedirs(output_dir, exist_ok=True)
    records = []

    ae_model.eval()
    unet_model.eval()
    noise_layer.eval()

    with torch.no_grad():
        for batch_idx, (imgs, labels, paths) in enumerate(loader):
            imgs = imgs.to(device)
            labels = labels.to(device)

            # 1. Encode latent vector
            z = ae_model.encoder(imgs)

            # 2. Generate watermarked images
            Iw = unet_model(imgs, z)

            # 3. Apply noise layer
            Iw_noisy = noise_layer(Iw)

            # 4. Save noisy images and latent vectors
            for i in range(imgs.size(0)):
                img_np = ((Iw_noisy[i].cpu().permute(1,2,0).numpy() * 0.5) + 0.5) * 255
                img_np = img_np.astype("uint8")
                save_path = os.path.join(output_dir, f"{batch_idx*loader.batch_size + i}.png")
                Image.fromarray(img_np).save(save_path)

                # Save latent vector as JSON string
                latent_json = json.dumps(z[i].cpu().tolist())
                records.append((save_path, labels[i].item(), latent_json))

    # Save CSV
    df = pd.DataFrame(records, columns=["path", "label", "latent"])
    df.to_csv(output_csv, index=False)
    print(f"✅ Noisy images saved to {output_dir} and CSV to {output_csv}")

# -----------------------------
# Usage
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataset = GTSRBDatasetCSV("./data/Train.csv", "./data/", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_dataset = GTSRBDatasetCSV("./data/Test.csv", "./data/", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Assume ae_model, unet_model, noise_layer are already loaded and on device
generate_noisy_images(
    loader=train_loader,
    ae_model=ae_model,
    unet_model=unet_model,
    noise_layer=noise_layer,
    output_dir="./data/NoisyWatermarkedTrain",
    output_csv="./data/NoisyWatermarkedTrain.csv",
    device=device
)
generate_noisy_images(
    loader=test_loader,
    ae_model=ae_model,
    unet_model=unet_model,
    noise_layer=noise_layer,
    output_dir="./data/NoisyWatermarkedTest",
    output_csv="./data/NoisyWatermarkedTest.csv",
    device=device
)

In [ ]:
import torch
import torch.nn as nn

class LatentDecoder(nn.Module):
    def __init__(self, latent_dim=128):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),  # 32x32 -> 16x16
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1), # 16x16 -> 8x8
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1), # 8x8 -> 4x4
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Flatten()
        )
        self.fc = nn.Linear(4*4*256, latent_dim)

    def forward(self, x):
        h = self.encoder(x)
        z_pred = self.fc(h)
        return z_pred


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import torch
import json

class NoisyLatentDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["path"]).convert("RGB")
        z = torch.tensor(json.loads(row["latent"]), dtype=torch.float32)  # JSON format
        if self.transform:
            img = self.transform(img)
        label = int(row["label"])
        return img, z, label


In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn

# -----------------------------
# Hyperparameters
# -----------------------------
latent_dim = 128
batch_size = 32
num_epochs = 10
lr = 1e-3

# -----------------------------
# Transforms & Datasets
# -----------------------------
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_dataset = NoisyLatentDataset("./data/NoisyWatermarkedTrain.csv", transform=transform)
test_dataset  = NoisyLatentDataset("./data/NoisyWatermarkedTest.csv", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# -----------------------------
# Model, Loss, Optimizer
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
decoder = LatentDecoder(latent_dim=latent_dim).to(device)
criterion = nn.MSELoss()  # latent consistency loss
optimizer = optim.Adam(decoder.parameters(), lr=lr)

# -----------------------------
# Training loop
# -----------------------------
best_loss = float("inf")
for epoch in range(num_epochs):
    decoder.train()
    running_loss = 0
    for imgs, z_target, _ in train_loader:
        imgs = imgs.to(device)
        z_target = z_target.to(device)

        optimizer.zero_grad()
        z_pred = decoder(imgs)
        loss = criterion(z_pred, z_target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)

    train_loss = running_loss / len(train_loader.dataset)

    # Evaluation
    decoder.eval()
    test_loss_total = 0
    with torch.no_grad():
        for imgs, z_target, _ in test_loader:
            imgs = imgs.to(device)
            z_target = z_target.to(device)
            z_pred = decoder(imgs)
            test_loss_total += criterion(z_pred, z_target).item() * imgs.size(0)

    test_loss = test_loss_total / len(test_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.6f} | Test Loss: {test_loss:.6f}")

    # Save best model
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(decoder.state_dict(), "./results/checkpoints/latent_decoder_best.pth")
        print(f"✅ Saved best latent decoder at epoch {epoch+1}")


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd

from models.autoencoder import Decoder       # Decoder class
# from latent_decoder import LatentDecoder     # Your trained latent decoder
from models.tsr_cnn import TSRNet           # Pretrained TSR model

# -----------------------------
# 1. Device & paths
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
noisy_csv = "./data/NoisyWatermarkedTest.csv"
batch_size = 32
latent_dim = 128

# -----------------------------
# 2. Dataset
# -----------------------------
class NoisyLatentTestDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["path"]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = int(row["label"])
        return img, label

# Normalization: same as TSR training
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
])

test_dataset = NoisyLatentTestDataset(noisy_csv, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# -----------------------------
# 3. Load models
# -----------------------------
# Latent decoder
latent_decoder = LatentDecoder(latent_dim=latent_dim).to(device)
latent_decoder.load_state_dict(torch.load("./results/checkpoints/latent_decoder_best.pth", map_location=device))
latent_decoder.eval()

# Autoencoder decoder
ae_decoder = Decoder(latent_dim=latent_dim).to(device)
ae_ckpt = torch.load("./results/checkpoints/autoencoder/autoencoder_best.pth", map_location=device)["model_state"]
decoder_state_dict = {k.replace("decoder.", ""): v for k, v in ae_ckpt.items() if k.startswith("decoder.")}
ae_decoder.load_state_dict(decoder_state_dict)
ae_decoder.eval()

# TSR model
tsr_model = TSRNet(num_classes=43).to(device)
tsr_ckpt = torch.load("./results/checkpoints/tsr_best_model.pth", map_location=device)
tsr_model.load_state_dict(tsr_ckpt["model_state_dict"])
tsr_model.eval()

# TSR normalization: [-1,1] -> [0,1], then normalize
tsr_normalize = transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))

# -----------------------------
# 4. Inference & Accuracy
# -----------------------------
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        # 1. Recover latent vector from noisy watermarked images
        z_pred = latent_decoder(imgs)

        # 2. Reconstruct images from latent vector
        recon_imgs = ae_decoder(z_pred)

        # 3. Scale to [0,1] then normalize for TSR
        recon_imgs = (recon_imgs + 1) / 2
        recon_imgs = tsr_normalize(recon_imgs)

        # 4. TSR predictions
        outputs = tsr_model(recon_imgs)
        _, preds = torch.max(outputs, 1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total * 100
print(f"🎯 TSR Accuracy on reconstructed images: {accuracy:.2f}%")
